<a href="https://colab.research.google.com/github/Dipak22/Case-Studies/blob/master/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create AlexNet from **scratch**



### Import necessary classes and functions

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from tqdm.notebook import tqdm
import numpy as np


In [20]:
class AlexNet(nn.Module):
  def __init__(self, classes=2, dropout_p = 0.5):
    super().__init__()
    self.classes = classes

    self.feature_extractor = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding =2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.BatchNorm2d(num_features=64),

        nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, stride=1, padding =2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.BatchNorm2d(num_features=192),

        nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, stride=1, padding =1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.BatchNorm2d(num_features=384),

        nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding =1),
        nn.ReLU(),
        nn.BatchNorm2d(num_features=256),

        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding =1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.BatchNorm2d(num_features=256),
    )

    self.avgpool = nn.AdaptiveAvgPool2d((6,6))

    self.head = nn.Sequential(
        nn.Dropout(dropout_p),
        nn.Linear(256*6*6, 4096),
        nn.ReLU(),
        nn.Dropout(dropout_p),
        nn.Linear(4096,4096),
        nn.ReLU(),
        nn.Linear(4096,classes)
    )
  def forward(self,x):
    batch_size = x.shape[0]

    x = self.feature_extractor(x)
    #print("1st shape", x.shape)
    x = self.avgpool(x)
    #print("2nd shape", x.shape)
    x = x.reshape(batch_size, -1)
    #print("3rd shape", x.shape)
    x = self.head(x)
    #print("4th shape", x.shape)
    return x


### get dogsvscat data

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("karakaggle/kaggle-cat-vs-dog-dataset")

print("Path to dataset files:", path)

100%|██████████| 787M/787M [00:07<00:00, 109MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/karakaggle/kaggle-cat-vs-dog-dataset/versions/1


In [9]:
PATH_TO_DATA = path +"/kagglecatsanddogs_3367a/PetImages/"

##Define transformations
normalizer = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    normalizer
])

dataset = ImageFolder(PATH_TO_DATA, transform=train_transforms)

train_samples, test_samples = int(0.9 * len(dataset)), len(dataset) - int(0.9 * len(dataset))

train_dataset, test_dataset = torch.utils.data.random_split(dataset=dataset, lengths=[train_samples, test_samples])

### Define hyperparameters and dataloaders

In [21]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = AlexNet()
model = model.to(DEVICE)

epochs = 5
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fn = nn.CrossEntropyLoss()
batch_size = 128

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

### define train method

In [ ]:
def train(model, device, epochs, optimizer, loss_fn, batch_size, trainloader, valloader):
    log_training = {"epoch": [],
                    "training_loss": [],
                    "training_acc": [],
                    "validation_loss": [],
                    "validation_acc": []}

    for epoch in range(1, epochs + 1):
        print(f"Starting Epoch {epoch}")
        training_losses, training_accuracies = [], []
        validation_losses, validation_accuracies = [], []

        model.train() # Turn On BatchNorm and Dropout
        for image, label in tqdm(trainloader):
            image, label = image.to(DEVICE), label.to(DEVICE)
            optimizer.zero_grad()
            out = model.forward(image)

            ### CALCULATE LOSS ##
            loss = loss_fn(out, label)
            training_losses.append(loss.item())

            ### CALCULATE ACCURACY ###
            predictions = torch.argmax(out, axis=1)
            accuracy = (predictions == label).sum() / len(predictions)
            training_accuracies.append(accuracy.item())

            loss.backward()
            optimizer.step()

        model.eval() # Turn Off Batchnorm
        for image, label in tqdm(valloader):
            image, label = image.to(DEVICE), label.to(DEVICE)
            with torch.no_grad():
                out = model.forward(image)

                ### CALCULATE LOSS ##
                loss = loss_fn(out, label)
                validation_losses.append(loss.item())

                ### CALCULATE ACCURACY ###
                predictions = torch.argmax(out, axis=1)
                accuracy = (predictions == label).sum() / len(predictions)
                validation_accuracies.append(accuracy.item())

        training_loss_mean, training_acc_mean = np.mean(training_losses), np.mean(training_accuracies)
        valid_loss_mean, valid_acc_mean = np.mean(validation_losses), np.mean(validation_accuracies)

        log_training["epoch"].append(epoch)
        log_training["training_loss"].append(training_loss_mean)
        log_training["training_acc"].append(training_acc_mean)
        log_training["validation_loss"].append(valid_loss_mean)
        log_training["validation_acc"].append(valid_acc_mean)

        print("Training Loss:", training_loss_mean)
        print("Training Acc:", training_acc_mean)
        print("Validation Loss:", valid_loss_mean)
        print("Validation Acc:", valid_acc_mean)

    return log_training, model


random_init_logs, model = train(model=model,
                                device=DEVICE,
                                epochs=epochs,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                batch_size=batch_size,
                                trainloader=trainloader,
                                valloader=valloader)